In [92]:
# make imports from pa_lib possible (parent directory of file's directory)
import sys
from pathlib import Path

file_dir = Path.cwd()
parent_dir = file_dir.parent
sys.path.append(str(parent_dir))

%load_ext autoreload
%autoreload

import pandas as pd
import numpy as np

from collections import namedtuple

from pa_lib.file import (
    store_bin,
    project_dir,
    data_files,
    load_csv,
    load_bin,
    write_xlsx,
    load_txt
)
from pa_lib.data import as_dtype, dtFactor, desc_col, lookup, clean_up_categoricals
from pa_lib.util import collect, value, flatten, normalize_rows, list_items
from pa_lib.log import time_log
from pa_lib.types import Record

# display long columns completely, show more rows
pd.set_option("display.max_colwidth", 300)
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load data files

In [93]:
with project_dir("MediaFocus"):
    display(data_files("*.feather"))
    br_kw = load_bin("branchen_kw_10y.feather")
    br_pg_kw = load_bin("branchen_pg_kw_10y.feather")
    br_wbt_kw = load_bin("branchen_wbt_kw.feather")

,size,mtime
name,,
branchen_kw_10y.feather,100.9 KB,31.10.19 15:28:44
branchen_pg_kw_10y.feather,553.2 KB,31.10.19 15:28:44
branchen_wbt_kw.feather,15.2 MB,31.10.19 15:28:44


2019-10-31 15:38:42 [INFO] Started loading binary file
2019-10-31 15:38:42 [INFO] Reading from file C:\Users\kpf\data\MediaFocus\branchen_kw_10y.feather
2019-10-31 15:38:42 [INFO] Finished loading binary file in 0.0s (0.0s CPU)
2019-10-31 15:38:42 [INFO] Started loading binary file
2019-10-31 15:38:42 [INFO] Reading from file C:\Users\kpf\data\MediaFocus\branchen_pg_kw_10y.feather
2019-10-31 15:38:42 [INFO] Finished loading binary file in 0.0s (0.0s CPU)
2019-10-31 15:38:42 [INFO] Started loading binary file
2019-10-31 15:38:42 [INFO] Reading from file C:\Users\kpf\data\MediaFocus\branchen_wbt_kw.feather
2019-10-31 15:38:42 [INFO] Finished loading binary file in 0.02s (0.19s CPU)


In [96]:
br_wbt_kw.head()

,Branche,Werbungtreibender,KW_01,KW_02,KW_03,KW_04,KW_05,KW_06,KW_07,KW_08,KW_09,KW_10,KW_11,KW_12,KW_13,KW_14,KW_15,KW_16,KW_17,KW_18,KW_19,KW_20,KW_21,KW_22,KW_23,KW_24,KW_25,KW_26,KW_27,KW_28,KW_29,KW_30,KW_31,KW_32,KW_33,KW_34,KW_35,KW_36,KW_37,KW_38,KW_39,KW_40,KW_41,KW_42,KW_43,KW_44,KW_45,KW_46,KW_47,KW_48,KW_49,KW_50,KW_51,KW_52,KW_53
0,"BAUEN, INDUSTRIE, EINRICHTUNG",16ART,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,960,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"BAUEN, INDUSTRIE, EINRICHTUNG",1A BRANDS HANDELS GMBH,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1280,1344,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1400,0,0,0,0,0,0,0,0
2,"BAUEN, INDUSTRIE, EINRICHTUNG",1A GSM AG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15000,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"BAUEN, INDUSTRIE, EINRICHTUNG",1A HUNKELER FENSTER AG,0,0,0,9562,0,0,2749,0,0,0,0,0,0,7613,0,5362,14384,7613,0,0,0,0,0,10245,0,10245,0,19923,7613,9904,0,17858,0,12836,6815,10020,0,0,0,9904,0,0,10724,0,0,10020,2749,46609,1490,45288,35148,0,0
4,"BAUEN, INDUSTRIE, EINRICHTUNG",1A HUNKELER HOLZBAU AG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10245,0,0,0,9923,0,0,0,0,0,13879,17516,0,9904,7613,10245,7702,17516,15577,0,0,0,9562,0,0,0,0,0,10020,0,0,0,7875,52695,13688,21625,0,0
